In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,  10, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(10, 10, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(10, 20, 3, padding=1)
        self.conv4 = nn.Conv2d(20, 20, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(20, 30, 3, padding=0)
        self.conv6 = nn.Conv2d(30, 10, 3)        
        self.avg_pool = nn.AvgPool2d(kernel_size=3, stride=3)

        self.batchNorm_1 = nn.BatchNorm2d(10)
        self.batchNorm_2 = nn.BatchNorm2d(10)
        self.batchNorm_3 = nn.BatchNorm2d(20)
        self.batchNorm_4 = nn.BatchNorm2d(20)
        self.batchNorm_5 = nn.BatchNorm2d(30)

        self.dropout = nn.Dropout(0.10) #just an experiemnt

    def forward(self, x):
        x = self.pool1(self.batchNorm_2(F.relu(self.conv2(self.batchNorm_1(F.relu(self.conv1(x)))))))
        x = self.dropout(x)
        x = self.pool2(self.batchNorm_4(F.relu(self.conv4(self.batchNorm_3(F.relu(self.conv3(x)))))))
        x = self.dropout(x)
        x = self.conv6(self.batchNorm_5(F.relu(self.conv5(x))))          
        x = self.avg_pool(x) 
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
            Conv2d-3           [-1, 10, 28, 28]             910
       BatchNorm2d-4           [-1, 10, 28, 28]              20
         MaxPool2d-5           [-1, 10, 14, 14]               0
           Dropout-6           [-1, 10, 14, 14]               0
            Conv2d-7           [-1, 20, 14, 14]           1,820
       BatchNorm2d-8           [-1, 20, 14, 14]              40
            Conv2d-9           [-1, 20, 14, 14]           3,620
      BatchNorm2d-10           [-1, 20, 14, 14]              40
        MaxPool2d-11             [-1, 20, 7, 7]               0
          Dropout-12             [-1, 20, 7, 7]               0
           Conv2d-13             [-1, 30, 5, 5]           5,430
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor()#,
                        #transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),#
                        #transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')       


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0492, Accuracy: 9848/10000 (98.48%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0358, Accuracy: 9891/10000 (98.91%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0185, Accuracy: 9940/10000 (99.40%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0178, Accuracy: 9942/10000 (99.42%)



loss=0.0126718170940876 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 51.12it/s]



Test set: Average loss: 0.0186, Accuracy: 9938/10000 (99.38%)

